# Feature engineering & selection

In [ ]:
import pandas as pd

TRAINING_SET_FILE = 'data/training_set.csv'

df = pd.read_csv(TRAINING_SET_FILE, sep=';')
df.T

In [ ]:
%matplotlib inline

# convertimos a datetimes
date_columns = ['fecha_reserva', 'check-in', 'check-out']
for d in date_columns:
    df[d] = pd.to_datetime(df[d])

df['reservation_days_ago'] = (df['check-in'] - df['fecha_reserva']).dt.days
df['total_noches'] = (df['check-out'] - df['check-in']).dt.days
df['precio_por_noche'] = df['importe_reserva'] / df['roomnigths']

groupby_country = df.groupby(by='pais_cliente')['is_reserva_cancelada']
mean_country_cancelations = groupby_country.sum() / groupby_country.count()

mean_country_cancelations = pd.DataFrame(mean_country_cancelations)\
    .rename(columns={'is_reserva_cancelada': 'pct_cancelacion_pais'})
mean_country_cancelations.reset_index(level=0, inplace=True)

df = df.merge(mean_country_cancelations, left_on='pais_cliente', right_on='pais_cliente')

#### Porcen Reser vs Cancelacion por pais

In [ ]:
res_por_pais = pd.pivot_table(df,index=["pais_cliente"],values=["index"], aggfunc=lambda x: len(x.unique()))
can_por_pais = pd.pivot_table(df,index=["pais_cliente"],values=["is_reserva_cancelada"], aggfunc=sum)
result = pd.concat([res_por_pais, can_por_pais], axis=1, join='inner',join_axes=[res_por_pais.index])
result["%"] = result["is_reserva_cancelada"]*100/result["index"]
result.sort_values("%", ascending=False)

# Feature selection

De todas las características que hemos construido, vamos a evaluar como funcionan.